In [1]:
import os
import sys
import numpy as np
import time
import argparse
import logging
import function_board as fb
import function_tool as ft
import cProfile
import function_get_aiming_grid
from function_solve_dp_withturn_server import solve_dp_withturn_valueiteration, setup_logging, log_system_info, \
    solve_dp_withturn_policyiteration
np.set_printoptions(precision=4)
np.set_printoptions(linewidth=300)
np.set_printoptions(threshold=300)
function_map = {
    'value': solve_dp_withturn_valueiteration,
    'policy': solve_dp_withturn_policyiteration
}
result_dir = '../HW_result'
if not os.path.isdir(result_dir):
    os.makedirs(result_dir)

In [2]:
iter_limit = 1000
iter_error = 1e-4
playerID_list = [7,11,12]

# cProfile

In [3]:
method = 'value'
for playerID in playerID_list:
    logger = setup_logging(method, result_dir, playerID)
    log_system_info(logger)
    name_pa = 'player{}'.format(playerID)
    [aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore,
     prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa)
    cProfile.run('''function_map[method](aiming_grid, prob_grid_normalscore,
                                                   prob_grid_doublescore,
                                                   prob_grid_bullscore, iter_limit=iter_limit,
                                                   iter_error=iter_error, logger=logger)''')

load_pickle from ../../data_parameter/player_gaussin_fit/grid_v2/player7_gaussin_prob_grid_v2.pkl
score_state = 501 finished.         9637428 function calls (9622632 primitive calls) in 41.476 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001   41.476   41.476 <string>:1(<module>)
      500    0.001    0.000    0.021    0.000 __init__.py:1056(flush)
      500    0.001    0.000    0.041    0.000 __init__.py:1067(emit)
      500    0.001    0.000    0.043    0.000 __init__.py:1174(emit)
      500    0.000    0.000    0.001    0.000 __init__.py:119(getLevelName)
      500    0.002    0.000    0.082    0.000 __init__.py:1412(debug)
      500    0.003    0.000    0.009    0.000 __init__.py:1502(findCaller)
      500    0.001    0.000    0.021    0.000 __init__.py:1538(makeRecord)
      500    0.002    0.000    0.079    0.000 __init__.py:1553(_log)
      500    0.001    0.000    0.001    0.000 __init__.p

KeyboardInterrupt: 

# ValueIteration

In [5]:
method = 'value'
for playerID in playerID_list:
    logger = setup_logging(method, result_dir, playerID)
    log_system_info(logger)
    name_pa = 'player{}'.format(playerID)
    [aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore,
     prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa)

    t1 = time.time()
    V, optimal_action_index = function_map[method](aiming_grid, prob_grid_normalscore,
                                                   prob_grid_doublescore,
                                                   prob_grid_bullscore, iter_limit=iter_limit,
                                                   iter_error=iter_error, logger=logger)
    t2 = time.time()

    print(f'\nsolve dp_withturn in {t2 - t1} seconds for player{playerID}')
    np.save(os.path.join(result_dir, f'player{playerID}_{method}iter_V_server.npy'), V)
    np.save(os.path.join(result_dir, f'player{playerID}_{method}iter_Mu_server.npy'), optimal_action_index)
    logger.info(f'Saved V matrix and optimal action index for player {playerID}')
    logger.info(f'Total time for {method} iteration: {t2 - t1:.4f} seconds.')
    print('\n')

load_pickle from ../../data_parameter/player_gaussin_fit/grid_v2/player7_gaussin_prob_grid_v2.pkl
score_state = 501 finished.
solve dp_withturn in 38.69153714179993 seconds for player7


# PolicyIteration

In [4]:
method = 'policy'
for playerID in playerID_list:
    logger = setup_logging(method, result_dir, playerID)
    log_system_info(logger)
    name_pa = 'player{}'.format(playerID)
    [aiming_grid, prob_grid_normalscore, prob_grid_singlescore, prob_grid_doublescore, prob_grid_triplescore,
     prob_grid_bullscore] = function_get_aiming_grid.load_aiming_grid(name_pa)

    t1 = time.time()
    V, optimal_action_index = function_map[method](aiming_grid, prob_grid_normalscore,
                                                   prob_grid_doublescore,
                                                   prob_grid_bullscore, iter_limit=iter_limit,
                                                   iter_error=iter_error, logger=logger)
    t2 = time.time()

    print(f'\nsolve dp_withturn in {t2 - t1} seconds for player{playerID}')
    np.save(os.path.join(result_dir, f'player{playerID}_{method}iter_V.npy'), V)
    np.save(os.path.join(result_dir, f'player{playerID}_{method}iter_Mu.npy'), optimal_action_index)
    logger.info(f'Saved V matrix and optimal action index for player {playerID}')
    logger.info(f'Total time for {method} iteration: {t2 - t1:.4f} seconds.')
    print('\n')

load_pickle from ../../data_parameter/player_gaussin_fit/grid_v2/player7_gaussin_prob_grid_v2.pkl
score_state = 501 finished.
solve dp_withturn in 0.3231353759765625 seconds for player7


load_pickle from ../../data_parameter/player_gaussin_fit/grid_v2/player11_gaussin_prob_grid_v2.pkl
score_state = 501 finished.
solve dp_withturn in 0.3071777820587158 seconds for player11
